# Sistema de controle do CTA pulsado

Este notebook tenta modelar o controle do anemômetro Pulsado. As variáveis do problema são listadas a seguir:

 * $T$ Temperatura do termistor
 * $R(T)$ Resistência elétrica do termistor a uma temperatura $T$
 * $T_w$ Temperatura do termistor desejada
 * $R_w$ Resistência de termistor na temperatura desejada
 * $R_i$ Resistência para medição da corrente
 * $R_{tot}$ Resistência total: $R(T) + R_i$
 * $E_i$ Tensão de alimentação
 * $x$ Parâmetro de controle do PWM ($0\le x \le 1$)
 * $I$ Corrente média passando pelo termistor
 * $T_a$ Temperatura ambiente
 * $h$ Coeficiente de convecção, depende da velocidade ($h = h(U)$)
 * $A$ Área externa do termistor
 
 ## Modelo do termoanemômetro
 
 Um balanço de energia, admitindo temperatura uniforme do termistor resulta na seguinte equação para a temperatura:
 
 $$
 mc_p\frac{dT}{dt} = -hA\cdot(T-T_a) + R(T)\cdot I^2
 $$
 
 Lembrando que se a frequência do PWM for alta de modo que o período seja muito menor que a constante de tempo do termistor (da ordem de 1s), podemos desprezar a dinâmica causada pelos pulsos e trabalhar com uma média que varia lentamente. Assim, 
 
 $$I = \frac{x\cdot E_i}{R(T) + R_i}$$
 
 Assim, 
 
 $$
 \frac{dT}{dt} = -\frac{h A}{m c_p}\left(T-T_a\right) + \frac{R(T)}{m c_p}\cdot \left(\frac{x E_i}{R{T} + R_i}\right)^2
 $$
 
 Esta equação pode ser reescrita como 
 $$
 \frac{dT}{dt} = -\beta(U)\left(T-T_a\right) + \eta x^2
 $$
onde
 $$\beta = \frac{h(U) A}{m c_p}$$
e
$$ \eta = \frac{R(T) E_i^2}{m c_p \left[R(T) + R_i\right]^2} $$

Como o objetivo é manter a temperatura (e portanto a resistência) constante e como $R(T) \gg R_i$ (concretamente, $R(20^\circ C) \approx 5 k\Omega$ $R(75^\circ C) \approx 1 k\Omega$ com $R_i\approx 0.1 k\Omega$) então pode-se admitir $\eta$ constante valendo
$$
\eta(T) \approx \eta(T_w) = \eta  = \frac{R_w E_i^2}{m c_p \left[R_w + R_i\right]^2} $$


## Controlador
Em um anemômetro de temperatura constante, o objetivo do circuito/programa do controlador é manter a temperatura do elemento sensor (termistor neste caso) constante com

$$
T = T_w
$$

Isto é feito variando o duty cycle do PWM da alimentação $x$. 

Definindo o erro como
$$\epsilon(t) = T(t) - T_w$$
em um controlador proporcional
$$
x(t) = \alpha_1\cdot \epsilon(t) - x_0
$$
É importante observar o parâmetro $x_0$. Em regime permanente, o erro vale zero mas é necessário que $x = x_0 > 0$ para manter o funcionamento. Ou seja, para que não haja erro em regime permanente ($\epsilon(t)=0$), é necessário que $x_0$ tenha  um valor que *depende da velocidade do vento*. Este valor pode ser calculado considerando $dT/dt = 0$:

$$
x_u = x(U) = \sqrt{\frac{\beta(U)\left(T_w-T_a\right)}{\eta}}
$$

A variável $x_u$ pode ser obtida durante a calibração mas durante o uso do anemômetro, este valor não é conhecido e fixando um valor de $x_0$ resulta em erro sistemático da temperatura que pode ser considerável. Assim, é necessário que exista algo para corrigir isso. Então será adotado um controlador Proporcional/Integral (PI):

$$
x(t) = \alpha_1\cdot\epsilon(t) + \alpha_2\cdot\int^t \epsilon(t)\:dt
$$

derivando esta equação pode-se chegar a uma equação diferencial para $x$:

$$
\frac{dx}{dt} = \alpha_1\frac{dT}{dt} + \alpha_2\cdot\left[T(t) - T_w\right]
$$


Assim temos um sistema de equações diferenciais ordinárias que podem ser resolvidas numericamente.


## Inicializar o ambiente julia

In [ ]:
include("../src/ThermistorHW.jl")
using ThermistorHW
using Plots
using CurveFit
using DifferentialEquations
gr()

In [ ]:
include("cta-pulsed.jl")

##  Montar o problema com valores típicos/esperados

In [ ]:
Rt = Thermistor(5000, 3000)
Tw = 85.0
cta = CTA.PulsedCTA(Rt)
Rw = Rt(Tw)
htrans = CTA.HeatTrans(Tw)
Ei = cta.Vi
Ri = cta.Ri
mcp = htrans.mcp
Ta = htrans.Ta

## Estimar $\beta(U)$ e $\eta$

In [ ]:
u = 0.2:0.1:20.0
β₁ = CTA.hamcpfun.(htrans, u, 0.0, 25);
β = CTA.hamcpfun.(htrans, u, 0.0, Tw);
etafun(R, mcp, Ri=100.0, Ei=24.0) = R * Ei^2 / (mcp * (R + Ri)^2 )
η = etafun(Rw, mcp, Ri, Ei)



In [ ]:
plot(u, β₁)
plot!(u, β)

In [ ]:
usqrt = sqrt.(u)
a1,a2, na = king_fit(sqrt.(β), u)
betafun(u) = a1 + a2 * u^na
plot(u, β)
plot!(u, betafun.(u))


In [ ]:
η

In [ ]:
xᵤ = @. sqrt( β * (Tw-Ta) / η )
b1,b2,nb = king_fit(sqrt.(xᵤ), u)
xfun(u) = b1 + b2*u^nb
plot(u, xᵤ)
plot!(u, xfun.(u))


In [ ]:
function efun(x, R, Ri=100.0, Ei=24.0)
        
    Eim = x * Ei
    
    I = Eim / (R + Ri)
    
    return I * Ri
end


## Controle para uma velocidade fixa

In [ ]:
function controle(du, u, p, t)
    Tw = p[1]
    β = p[2]
    η = p[3]
    α₁ = p[4]
    α₂ = p[5]
    dT = -β*u[1] + η*u[2]^2
    dx = -α₁*dT - α₂*(u[1]-Tw)
    du[1] = dT
    du[2] = dx
end

In [ ]:

u0 = [0.0, 0.5]
tspan = (0.0, 200.0)
p = (50.0, 0.22, η, 0.00015, 0.00012)
prob = ODEProblem(controle, u0, tspan, p)

In [ ]:
s = solve(prob);

plot(s.t, s[1,:]./50.0)
plot!(s.t, s[2,:])

## Mudando a velocidade

In [ ]:
function ctacontrole(du, u, p, t)
    
    cta = p[1]
    htrans = p[2]
    Tw = htrans.Tw
    vel = p[3]
    Ta = htrans.Ta
    
    R = cta.Rt(u[1])
    η = etafun(R, htrans.mcp, cta.Ri, cta.Vi)
    V = vel(t)
    β = CTA.hamcpfun(htrans, V, t, u[1])
    
    α₁ = p[4]
    α₂ = p[5]

    dT = -β*(u[1]-Ta) + η*u[2]^2
    dx = -α₁*dT - α₂*(u[1]-Tw)
    
    du[1] = dT
    du[2] = dx
end

### Definição do Problema

In [ ]:
Rt1 = Thermistor(5000, 3000)
Tw1 = 85.0
Rw1 = Rt1(Tw1)
cta1 = CTA.PulsedCTA(Rt1)
htrans1 = CTA.HeatTrans(Tw1)
Ta1 = htrans.Ta


### Curva de Calibração

In [ ]:
ucal = 0.3:0.1:20.0
etacal = etafun(Rt1(Tw1), htrans1.mcp, cta1.Ri, cta1.Vi)
betacal = CTA.hamcpfun.(htrans1, ucal, 0.0, Tw1)
xcal = @. sqrt(betacal*(Tw1-Ta1) / etacal)
Ecal = efun.(xcal, Rw1, cta1.Ri, cta1.Vi)

xcalibr = KingFit(Ecal, ucal)

plot(ucal, Ecal)
plot!(xcalibr.(Ecal), Ecal)


In [ ]:
xcalibr(1.5)

In [ ]:
#vel = CTA.VelRampConst()
vel = CTA.VelFreq(10.0, 50, 4)
#vel = CTA.VelStep(10, 0, 10)
p = (cta1, htrans1, vel, 15, 15)

T0 = Tw1
eta0 = etafun(Rt1(T0), htrans1.mcp, cta1.Ri, cta1.Vi)
beta0 = CTA.hamcpfun(htrans1, vel(0.0), 0.0, T0)
x0 = sqrt(beta0*(T0-Ta1)/eta0)

#Equação de calibração:

u0 = [Tw1, x0]

tspan = (0.0, 1.0)



prob2 = ODEProblem(ctacontrole, u0, tspan, p)
s = solve(prob2, dtmax=0.01)
T = s[1,:]
x = s[2,:]
Eo = efun.(x, Rt.(T))
Uc = xcalibr.(Eo)
#plot(s.t, s[1,:]./50.0)
plot(s.t, vel.(s.t))
plot!(s.t, Uc)


In [ ]:
plot(s.t, s[2,:])

## Controle digital

Na seção anterior chegamos a equações para modelar o termistor e o sistema de controle. A próxima etapa é discretizar isso, chegando a algo que possa ser implementado em um microcontrolador.

Usando diferenças finitas, e chamando

$$
\hat{T}_i = -\beta\left(T_i - T_a\right) + \eta x_i^2
$$

chega-se à seguinte equação para $T_i$:

$$
    T_{i+1} = T_i + \Delta t \cdot\left[ -\beta\left(T_i - T_a\right) + \eta x_i^2\right] = T_i + \Delta t\cdot\hat{T}_i
$$

A equação para $x$ é:

$$
x_{i+1} = x_i + \Delta t\cdot\left[\alpha_1\hat{T}_i + \alpha_2\left(T_i-T_w\right)\right]
$$

In [ ]:
function ctadigital(x, T, t, U, dt, cta, htrans, α₁=0.5, α₂=0.5,  σ=0.0)
  
    Tw = htrans.Tw
    Ta = htrans.Ta
    eT = σ*randn()
    T1 = T + eT
    
    R = cta.Rt(T1)
    η = etafun(R, htrans.mcp, cta.Ri, cta.Vi)
    β = CTA.hamcpfun(htrans, U, t, T1)
    
    That = -β*(T1-Ta) + η*x^2
    
    return x - dt*(α₁*That + α₂*(T1 - Tw)), T + dt*That
end


### Definição do problema


In [ ]:

Rt2 = Thermistor(5000, 3000)
Tw2 = 85.0
Rw2 = Rt1(Tw2)
cta2 = CTA.PulsedCTA(Rt2)
htrans2 = CTA.HeatTrans(Tw2)
Ta2 = htrans.Ta



### Nova curva de calibração

In [ ]:
ucal2 = 0.3:0.1:20.0
etacal2 = etafun(Rt2(Tw2), htrans2.mcp, cta2.Ri, cta2.Vi)
betacal2 = CTA.hamcpfun.(htrans2, ucal2, 0.0, Tw2)
xcal2 = @. sqrt(betacal2*(Tw2-Ta2) / etacal2)
Ecal2 = efun.(xcal2, Rw2, cta2.Ri, cta2.Vi)

xcalibr2 = KingFit(Ecal2, ucal2)



In [ ]:
dt = 0.02

#vel = CTA.VelRampConst()
vel = CTA.VelFreq(10.0, 0.1, 4)

T0 = Tw2
eta0 = etafun(Rt2(T0), htrans2.mcp, cta2.Ri, cta2.Vi)
beta0 = CTA.hamcpfun(htrans2, vel(0.0), 0.0, T0)
x0 = sqrt(beta0*(T0-Ta1)/eta0)

tmax = 60.0

#Equação de calibração:

u0 = [T0, x0]

tspan = (0.0, 60.0)

tt = 0.0:dt:tmax
nt = length(tt)
T = zeros(nt)
x = zeros(nt)


T[1] = T0
x[1] = x0

for i  = 2:nt
    U =  vel(tt[i])
    x[i], T[i] = ctadigital(x[i-1], T[i-1], tt[i], U, dt, cta, htrans, 0.03, 0.02, 5.0)        
end

Eo = efun.(x, Rt2.(T))
Uc = xcalibr2.(Eo)

plot(tt, Uc)
plot!(tt, vel.(tt))


In [ ]:
plot(tt,Eo)